# import packages

In [ ]:
library(reticulate)
use_condaenv("mar25", required = TRUE)


In [ ]:
library(Signac)
library(Seurat)
library(data.table)
library(ggplot2)
library(tidyverse)
library(readxl)
library(patchwork)
library(ComplexHeatmap)
library(patchwork)
library(ggpubr)
library(GenomicRanges) 
library(EnsDb.Mmusculus.v79)
library(DT)
library(future)
library(anndata)

plan("multicore", workers = 4)

options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM

#base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_JinWang/'
base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/repos/niche/'
setwd(base_dir)


# import design table

In [ ]:
design = fread(file = 'data/dsn_dt_base_v4.tsv')


In [ ]:
ma9_msc_samples = c('M2', 'M3', 'M4', 'M4_m10', 'ctrl_m4w', 'ctrl_m6w', 'ctrl_m8w')

msc_design = design[sample_name %in% ma9_msc_samples,] 
msc_design


# import MSC RNAseq analysis

## import all data

In [ ]:
# h5 files were generated by cellranger
h5_list = lapply(msc_design$h5file, Read10X_h5)

In [ ]:
# create seurat object using the gene expression data
rna_seu_list = lapply(1:nrow(msc_design), function(ii){
    h5 = h5_list[[ii]]
    sampleID2 = msc_design$sampleID2[ii]
    oo = CreateSeuratObject(counts = h5$`Gene Expression`, assay = 'RNA', project = sampleID2)
    RenameCells(object = oo, new.names = paste0(sampleID2, '__', Cells(oo)))
})


In [ ]:
# create seurat object using the atac sequencing data
ata_seu_list = lapply(1:nrow(msc_design), function(ii){
    h5 = h5_list[[ii]]
    sampleID2 = msc_design$sampleID2[ii]
    oo = CreateSeuratObject(counts = h5$`Peaks`, assay = 'ATAC', project = sampleID2)
    RenameCells(object = oo, new.names = paste0(sampleID2, '__', Cells(oo)))
})
# CreateAssay5Object(counts = pbmc.counts)

In [ ]:
# merge all samples
rna_seu  = merge(rna_seu_list[[1]], rna_seu_list[2:length(rna_seu_list)], labels = msc_design$sampleID2)
ata_seu  = merge(ata_seu_list[[1]], ata_seu_list[2:length(ata_seu_list)], labels = msc_design$sampleID2)


In [ ]:
rna_seu
ata_seu

In [ ]:
# checking cell meta data
tmp = data.table(id = Cells(rna_seu))
tmp[, sampleID := sub('__.*', '', id)] 
tmp[, cellID_n := sub('.*__', '', id)] 
tmp[, cellID   := sub('_.*', '', cellID_n)] 
tmp[, n := sub('.*_', '', cellID_n)] 
head(tmp)
table(tmp$sampleID, tmp$n)


## select RNAseq based on previous analysis

In [ ]:
# import previous data only for MSC cell IDs 
#  ~ 3900 cells
msc_rna_2 = read_h5ad("data/msc_rna_2_mar12.h5ad") 

In [ ]:
rna_ids = as.data.table(msc_rna_2$obs, keep.rownames = T)
rna_ids[, id := rn] 
head(rna_ids)

In [ ]:
rna_ids[, cell_ID := sub('_.*', '', id) ]
rna_ids[, sample_ID := sub('.*_', '', id) ]
table(rna_ids$sample_ID)

In [ ]:
rna_ids[, cell_ID := sub('_.*', '', id) ]
rna_ids[, sample_ID := sub('.*_', '', id) ]
rna_ids[, sample_ID2 := sample_ID]
rna_ids[sample_ID == 'm10', sample_ID2 := 'M4']

rna_ids[sample_ID2 == 'M2', sampleID := 'MA9_MSC_wk4']
rna_ids[sample_ID2 == 'M3', sampleID := 'MA9_MSC_wk6']
rna_ids[sample_ID2 == 'M4', sampleID := 'MA9_MSC_wk8']
rna_ids[sample_ID2 == 'm4w', sampleID := 'Ctrl_MSC_wk4']
rna_ids[sample_ID2 == 'm6w', sampleID := 'Ctrl_MSC_wk6']
rna_ids[sample_ID2 == 'm8w', sampleID := 'Ctrl_MSC_wk8']

rna_ids[sample_ID == 'M2',  nn := 1 ]
rna_ids[sample_ID == 'M3',  nn := 2 ]
rna_ids[sample_ID == 'M4',  nn := 3 ]
rna_ids[sample_ID == 'm10', nn := 4 ]
rna_ids[sample_ID == 'm4w', nn := 5 ]
rna_ids[sample_ID == 'm6w', nn := 6 ]
rna_ids[sample_ID == 'm8w', nn := 7 ]

rna_ids[, seuratID := paste0(sampleID, '__', cell_ID, '_', nn)]
head(rna_ids)


In [ ]:
# subset cells based on these IDs
rna_seu_2 = subset(rna_seu, cells = Cells(rna_seu)[Cells(rna_seu) %in% rna_ids$seuratID])

# atac seq will be processed by importing raw data below
#ata_seu_2 = subset(ata_seu, cells = Cells(ata_seu)[Cells(ata_seu) %in% rna_ids$seuratID])

## preprocess for selected RNA 

In [ ]:
seurat_obj <- FindVariableFeatures(seurat_obj, selection.method = "vst", nfeatures = 2000)


In [ ]:
rna_seu_2 <- SCTransform(rna_seu_2, verbose = FALSE, return.only.var.genes = F)
rna_seu_2 <- RunPCA(rna_seu_2, verbose = FALSE)
rna_seu_2 <- RunUMAP(rna_seu_2, dims = 1:30, verbose = FALSE)

rna_seu_2 <- FindNeighbors(rna_seu_2, dims = 1:30, verbose = FALSE)
rna_seu_2 <- FindClusters(rna_seu_2, verbose = FALSE, resolution = .2)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)
DimPlot(rna_seu_2, group.by = c("orig.ident", "seurat_clusters"), label = TRUE) + ggtitle("RNA") 


In [ ]:
rna_seu_2$umap1 = as.data.frame(Embeddings(rna_seu_2, 'umap'))$umap_1
rna_seu_2$umap2 = as.data.frame(Embeddings(rna_seu_2, 'umap'))$umap_2

In [ ]:
rna_seu_3 <- subset(rna_seu_2, idents = c(0, 1, 2))


In [ ]:
rna_seu_3

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)
DimPlot(rna_seu_3, group.by = c("orig.ident", "seurat_clusters"), label = TRUE) + ggtitle("RNA") 


In [ ]:
# add genome information 
# annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
# seqlevelsStyle(annotations) <- "UCSC"
# genome(annotations) <- "mm10"
# Annotation(ata_seu_2) <- annotations 
# ata_seu_3 = subset(ata_seu_2, cells = Cells(rna_seu_3)) 
# We exclude the first dimension as this is typically correlated with sequencing depth
# DefaultAssay(ata_seu_3) <- "ATAC"
# ata_seu_3 <- RunTFIDF(ata_seu_3, assay = "ATAC")
# ata_seu_3 <- FindTopFeatures(ata_seu_3, min.cutoff = "q0")
# ata_seu_3 <- RunSVD(ata_seu_3)
# ata_seu_3 <- RunUMAP(ata_seu_3, reduction = "lsi", dims = 2:30, reduction.name = "umap.atac", reduction.key = "atacUMAP_", assay = "ATAC")



In [ ]:
ata_seu_3 = subset(ata_seu_2, cells = Cells(rna_seu_3)) 


# exported files

In [ ]:
# saveRDS(ata_seu, file = 'data/ata_seu.rds') 
# saveRDS(ata_seu_5, file = 'data/ata_seu_5.rds') 
# saveRDS(cds, file = 'data/cds_monocle3.rds') 
# saveRDS(oo_msc_5, file = 'data/oo_msc_5.rds')
# saveRDS(oo_msc_5_peaks, file = 'data/oo_msc_5_peaks.rds')
# saveRDS(oo_msc_5_peaks_4clusters, file = 'data/oo_msc_5_peaks_4clusters.rds')
# saveRDS(rna_seu, file = 'data/rna_seu.rds') 
# saveRDS(rna_seu_5, file = 'data/rna_seu_5.rds') 
# saveRDS(rna_seu_5_markers, file = 'data/rna_seu_5_markers.rds')


# import ATACSeq 

## import raw data

In [ ]:
atac_peak_list = lapply(msc_design$peak_file, function(fname){
    tmp = read.table(fname, col.names = c("chr", "start", "end"))
    makeGRangesFromDataFrame(tmp)
})

In [ ]:
atac_peaks <- Signac::reduce(c(
    atac_peak_list[[1]], atac_peak_list[[2]], atac_peak_list[[3]], atac_peak_list[[4]],
    atac_peak_list[[5]], atac_peak_list[[6]], atac_peak_list[[7]]))


In [ ]:
# Filter out bad peaks based on length
peakwidths <- width(atac_peaks)
atac_peaks <- atac_peaks[peakwidths  < 10000 & peakwidths > 20]
atac_peaks


In [ ]:
metrics_list = lapply(msc_design$metric_file, function(fname){
    tmp <- read.table(fname, stringsAsFactors = FALSE, sep = ",", header = TRUE, row.names = 1 )
    tmp[-1, ]
})

In [ ]:
summary(metrics_list[[1]]$atac_raw_reads)

In [ ]:
dim(metrics_list[[1]][metrics_list[[1]]$atac_raw_reads > 1500, ])

In [ ]:
mtrics_filtered_list = lapply(metrics_list, function(tmp){tmp[tmp$atac_raw_reads > 1000, ]})

In [ ]:
lapply(mtrics_filtered_list, nrow)

In [ ]:
msc_frag_list = lapply(1:length(mtrics_filtered_list), function(ii){ 
    fname = msc_design[ii, fragment_file]
    cells = mtrics_filtered_list[[ii]][, 'gex_barcode']
    CreateFragmentObject( path = fname, cells = cells) }
)

In [ ]:
feature_counts_list = lapply(1:length(msc_frag_list), function(ii){
    frag  = msc_frag_list[[ii]]
    cells = mtrics_filtered_list[[ii]][, 'gex_barcode']
    FeatureMatrix( fragments = frag, features = atac_peaks, cells = cells)}  )

In [ ]:
atac_assay_list = lapply(1:length(feature_counts_list), function(ii){
    CreateChromatinAssay(feature_counts_list[[ii]], fragments = msc_frag_list[[ii]])
})

In [ ]:
atac_oo_list = lapply(1:length(atac_assay_list), function(ii){
    oo = CreateSeuratObject(atac_assay_list[[ii]], assay = 'ATAC', meta.data = msc_frag_list[[ii]])
    oo$dataset = msc_design$sample_name
    oo
})

In [ ]:
msc_design$sampleID2 = sub('r2', '', msc_design$sampleID)

In [ ]:
msc_design$sampleID2

## merge ATAC

In [ ]:
oo <- merge(
  x = atac_oo_list[[1]],
  y = atac_oo_list[2:length(atac_oo_list)],
  add.cell.ids = paste0(msc_design$sampleID2, '_')
)

In [ ]:
oo[["ATAC"]]

## filter MSC ATAC sequencing cells based on RNASeq selected cells

In [ ]:
msc_dt = data.table(id = names(oo$dataset))
msc_dt[, sample_ID := sub('__.*', '', id) ]
msc_dt[, cell_ID := sub('.*__(.*)_.*', '\\1', id)]
head(msc_dt) 


In [ ]:
table(msc_dt$sample_ID)

In [ ]:
table(rna_ids$sample_ID2)

In [ ]:
length(intersect(msc_dt$id, rna_ids$seuratID))

In [ ]:
msc_dt[, ov := F]
msc_dt[id %in% rna_ids$seuratID, ov := T]


In [ ]:
head(msc_dt[ov == T, ])
nrow(msc_dt[ov == T, ])

In [ ]:
oo@meta.data$ov = msc_dt$ov

In [ ]:
oo_msc = subset(oo, ov == T)

In [ ]:
oo_msc

In [ ]:
oo_msc <- RunTFIDF(oo_msc)
oo_msc <- FindTopFeatures(oo_msc, min.cutoff = 20)
oo_msc <- RunSVD(oo_msc)
oo_msc <- RunUMAP(oo_msc, dims = 2:50, reduction = 'lsi')
set.seed(1234)
oo_msc <- FindNeighbors(oo_msc, dims = 2:50, reduction = "lsi")
set.seed(1234)
oo_msc <- FindClusters(oo_msc, resolution = 0.5)  # Adjust resolution as needed


In [ ]:
DimPlot(oo_msc, reduction = "umap", label = TRUE)


In [ ]:
oo_msc@meta.data$umap1 = as.data.frame(Embeddings(oo_msc, reduction = 'umap'))$umap_1
oo_msc@meta.data$umap2 = as.data.frame(Embeddings(oo_msc, reduction = 'umap'))$umap_2


In [ ]:
rm <- subset(oo_msc, umap1 > 4 & umap2 < -3)
oo_msc_2 = subset(oo_msc, cells = setdiff(Cells(oo_msc), Cells(rm)))


In [ ]:
oo_msc_2 <- RunTFIDF(oo_msc_2)
oo_msc_2 <- FindTopFeatures(oo_msc_2, min.cutoff = 20)
oo_msc_2 <- RunSVD(oo_msc_2)
oo_msc_2 <- RunUMAP(oo_msc_2, dims = 2:50, reduction = 'lsi')
set.seed(1234)
oo_msc_2 <- FindNeighbors(object = oo_msc_2, reduction = 'lsi', dims = 2:30)
set.seed(1234)
oo_msc_2 <- FindClusters(object = oo_msc_2, verbose = FALSE, algorithm = 3, resolution = .1)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)
DimPlot(rna_seu_2, group.by = c("orig.ident", "seurat_clusters"), label = TRUE) + ggtitle("RNA") 


In [ ]:
rna_seu_2$umap1 = as.data.frame(Embeddings(rna_seu_2, 'umap'))$umap_1
rna_seu_2$umap2 = as.data.frame(Embeddings(rna_seu_2, 'umap'))$umap_2

In [ ]:
rna_seu_3 = subset(rna_seu_2, subset =  umap1 > -5)

In [ ]:
oo_msc_3 = subset(oo_msc_2, cells = Cells(rna_seu_3)) 

In [ ]:
ov_cells = intersect(Cells(oo_msc_2), Cells(rna_seu_2))
length(ov_cells)

oo_msc_3 = subset(oo_msc_2, cells = ov_cells) 
rna_seu_3 = subset(rna_seu_2, cells = ov_cells) 


In [ ]:
DefaultAssay(oo_msc_3) = 'ATAC'

In [ ]:
oo_msc_3 <- RunTFIDF(oo_msc_3)
oo_msc_3 <- FindTopFeatures(oo_msc_3, min.cutoff = 20)
oo_msc_3 <- RunSVD(oo_msc_3)
oo_msc_3 <- RunUMAP(oo_msc_3, dims = 2:50, reduction = 'lsi')
set.seed(1234)
oo_msc_3 <- FindNeighbors(object = oo_msc_3, reduction = 'lsi', dims = 2:30)
set.seed(1234)
oo_msc_3 <- FindClusters(object = oo_msc_3, verbose = FALSE, algorithm = 3, resolution = .2)


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(oo_msc_3, group.by = c('seurat_clusters'), pt.size = 0.1)


In [ ]:
oo_msc_4 = subset(oo_msc_3, subset = seurat_clusters %in% c(0,1,2,3,4))

In [ ]:
oo_msc_4 <- RunTFIDF(oo_msc_4)
oo_msc_4 <- FindTopFeatures(oo_msc_4, min.cutoff = 20)
oo_msc_4 <- RunSVD(oo_msc_4)
oo_msc_4 <- RunUMAP(oo_msc_4, dims = 2:50, reduction = 'lsi')
set.seed(1234)
oo_msc_4 <- FindNeighbors(object = oo_msc_4, reduction = 'lsi', dims = 2:30)
set.seed(1234)
oo_msc_4 <- FindClusters(object = oo_msc_4, verbose = FALSE, algorithm = 3, resolution = .2)


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(oo_msc_4, group.by = c('seurat_clusters'), pt.size = 0.1)


In [ ]:
set.seed(1234)
rna_seu_3 <- FindClusters(object = rna_seu_3, verbose = FALSE, algorithm = 3, resolution = .2)

options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(rna_seu_3, group.by = c('orig.ident', 'seurat_clusters'), pt.size = 0.1)


In [ ]:

options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(rna_seu_3, group.by = c('seurat_clusters'), pt.size = 0.1) + geom_vline(xintercept = -3.4, color = 'grey70')


In [ ]:
rna_seu_3@meta.data$umap1 = as.data.frame(Embeddings(rna_seu_3, reduction = 'umap'))$umap_1
rna_seu_3@meta.data$umap2 = as.data.frame(Embeddings(rna_seu_3, reduction = 'umap'))$umap_2


In [ ]:
#rna_seu_4 = subset(rna_seu_3, subset = seurat_clusters %in% c(0, 1, 2))
rna_seu_4 = subset(rna_seu_3, subset = umap1 > -3.4)

In [ ]:
ov_cells_4 = intersect(Cells(oo_msc_4), Cells(rna_seu_4))
oo_msc_5 = subset(oo_msc_4, cells = ov_cells_4)
rna_seu_5 = subset(rna_seu_4, cells = ov_cells_4)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(oo_msc_5, group.by = c('sampleID', 'seurat_clusters'), pt.size = 0.1)


# re-organize RNASeq datasets

In [ ]:
# to get consistent clusters, an better version were exported from previous run
rna_seu_5 = readRDS('data/rna_seu_5.rds')


In [ ]:
rna_seu_5@meta.data$umap1 = as.data.frame(Embeddings(rna_seu_5, reduction = 'umap'))$umap_1
rna_seu_5@meta.data$umap2 = as.data.frame(Embeddings(rna_seu_5, reduction = 'umap'))$umap_2


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 200)
DimPlot(rna_seu_5, label = TRUE, group.by = c('orig.ident', 'seurat_clusters_adj'))

In [ ]:
rna_seu_5$seurat_clusters[rna_seu_5$umap2 < 0.5 & rna_seu_5$seurat_clusters == '5'] = '0' 
table(rna_seu_5$seurat_clusters)

In [ ]:
# re-organize RNASeq cluster names
rna_seu_5$seurat_clusters_adj = as.character(rna_seu_5$seurat_clusters)
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '1'] = '1'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '6'] = '2'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '3'] = '3'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '0'] = '4'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '5'] = '5'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '4'] = '6'
rna_seu_5$seurat_clusters_adj[rna_seu_5$seurat_clusters == '2'] = '7'
rna_seu_5$seurat_clusters_adj = factor(rna_seu_5$seurat_clusters_adj , levels = c('1', '2', '3', '4', '5', '6', '7'))


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 200)
DimPlot(rna_seu_5, label = TRUE, group.by = c('orig.ident', 'seurat_clusters_adj'))

In [ ]:
table(rna_seu_5$seurat_clusters_adj)

# re-organize ATAC-Seq data


In [ ]:
oo_msc_5 = readRDS(file = 'data/oo_msc_5.rds')

In [ ]:
oo_msc_5

In [ ]:
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "mm10"
Annotation(oo_msc_5) = annotations

In [ ]:
oo_msc_5$seurat_clusters_adj = rna_seu_5@meta.data[Cells(oo_msc_5), 'seurat_clusters_adj']

In [ ]:
oo_msc_5$sampleID = sub('__.*', '', Cells(oo_msc_5))

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 4, repr.plot.res = 300)
DimPlot(oo_msc_5, group.by = c('sampleID', 'seurat_clusters', 'seurat_clusters_adj'), pt.size = 0.1)
# sample ID, scATAC-Seq clusters, scRNA-Seq clusters

# plot sample vs cell numbers in each of the 4 clusters

In [ ]:
plotdat = as.data.table(table(oo_msc_5$sampleID, oo_msc_5$seurat_clusters_adj))
setnames(plotdat, c('sampleID', 'cluster', 'N'))
plotdat[, cluster := factor(cluster)]
head(plotdat)


In [ ]:
plotdat[N < 10, N := 0]

In [ ]:
options(repr.plot.res = 300, repr.plot.height = 4, repr.plot.width = 6)
ggplot(plotdat, aes(x = cluster, y = N,  fill = sampleID)) + 
geom_bar(stat = 'identity', position = 'dodge') + theme_pubr() +
ylab('Number of cells')

# call peaks and plot

## can skip this subsetion, need load oo_msc_5 in the next subsetion

In [ ]:
# saveRDS(oo_msc_5_peaks, file = 'data/oo_msc_5_peaks.rds')
# oo_msc_5_peaks = readRDS(file = 'data/oo_msc_5_peaks.rds')
oo_msc_5_peaks = CallPeaks(oo_msc_5, group.by = 'seurat_clusters_adj', macs2.path = '~/conda3/envs/mar25/bin/macs3')


In [ ]:
oo_msc_5_peaks

In [ ]:
# organize clusters for peak
oo_msc_5_peaks_dt = as.data.table(oo_msc_5_peaks)
oo_msc_5_peaks_dt[, peak_called_in := paste(sort(unlist(strsplit(.SD$peak_called_in, split = ','))), collapse=','), by = 1:nrow(oo_msc_5_peaks_dt)]
head(oo_msc_5_peaks_dt)


In [ ]:
oo_msc_5_peaks_dt[seqnames == 'chr3' & start > 90600391 & end < 90615832, 1:3]

In [ ]:
# combine some of the clusters since these are very close WT samples
oo_msc_5$msc_4clusters = 'cl_other'
oo_msc_5$msc_4clusters[oo_msc_5$seurat_clusters_adj %in% c('5')]  = 'cl_5'
oo_msc_5$msc_4clusters[oo_msc_5$seurat_clusters_adj %in% c('6')]  = 'cl_6'
oo_msc_5$msc_4clusters[oo_msc_5$seurat_clusters_adj %in% c('7')]  = 'cl_7'
table(oo_msc_5$msc_4clusters)


In [ ]:
SplitFragments( oo_msc_5, assay = 'ATAC', group.by = 'seurat_clusters_adj',
  outdir = getwd(), file.suffix = "oo_msc_5_splitfragment", outdir = 'data/', 
  append = TRUE, buffer_length = 256L, verbose = TRUE )

In [ ]:
# Re-call peaks using MACS3 based on cluster information  
# This is a time-consuming step  

# Save the results for future use  
# saveRDS(oo_msc_5_peaks_4clusters, file = 'data/oo_msc_5_peaks_4clusters.rds')  

# Load previously saved results if needed  
# oo_msc_5_peaks_4clusters <- readRDS(file = 'data/oo_msc_5_peaks_4clusters.rds')  

# Perform peak calling with MACS3, grouping by 'msc_4clusters'  
oo_msc_5_peaks_4clusters <- CallPeaks(  
  object = oo_msc_5,  
  macs2.path = '~/conda3/envs/mar25/bin/macs3',  
  group.by = 'msc_4clusters'  
)  


In [ ]:
# saveRDS(oo_msc_5, file = 'data/oo_msc_5_w_info.rds')
# saveRDS(oo_msc_5_peaks_dt, file = 'data/oo_msc_5_peaks_dt.rds')
# saveRDS(oo_msc_5_peaks, file = 'data/oo_msc_5_peaks.rds')
# saveRDS(oo_msc_5_peaks_4clusters, file = 'data/oo_msc_5_peaks_4clusters.rds')


## explore the peaks (run the comment cell below if needed)

In [ ]:
# run these step if skipped the previous subsection
# oo_msc_5 = readRDS(file = 'data/oo_msc_5_w_info.rds')
# oo_msc_5_peaks_dt = readRDS(file = 'data/oo_msc_5_peaks_dt.rds')
# oo_msc_5_peaks = readRDS(file = 'data/oo_msc_5_peaks.rds')
# oo_msc_5_peaks_4clusters = readRDS(file = 'data/oo_msc_5_peaks_4clusters.rds')

In [ ]:
oo_msc_5@assays$ATAC

In [ ]:
# checking functions of this peak in other assays
# chr3 90604391-90604832 in mm10
subsetByOverlaps(oo_msc_5_peaks, GRanges(seqnames = 'chr3', ranges = IRanges(start = 90600882, end = 90614414)))

In [ ]:
oo_msc_5_peaks[13302, ]

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'sampleID',
  region = "chr3-90600882-90614414"
)
# S100a6 chr3:90,612,882-90,614,414


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters',
  region = "chr3-90604391-90604832"
)
# S100a6 chr3:90,612,882-90,614,414


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters_adj',
  region = "chr3-90604391-90604832"
)
# S100a6 chr3:90,612,882-90,614,414


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters_adj',
  region = "chr3-90600882-90614414"
)
# S100a6 chr3:90,612,882-90,614,414


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters_adj',
  region = "chr1-4767454-4769671"
)


In [ ]:
oo_msc_5@assays$RNA2 = rna_seu_5@assays$RNA

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3, repr.plot.res = 200)
CoveragePlot(
    object = oo_msc_5,
    #features = c('S100a1', 'S100a13', 'S100a16', 'S100a4', 'S100a6', 'S100a7', 'S100a8', 'S100a9'), 
    expression.assay = "RNA2",
    expression.slot = "data",
    group.by = 'seurat_clusters_adj',
    region = "chr3-90600882-90614414"
)
# S100a6 chr3:90,612,882-90,614,414
# features = c('S100a1', 'S100a13', 'S100a14', 'S100a16', 'S100a3', 'S100a4', 'S100a5', 'S100a6', 'S100a7', 'S100a8', 'S100a9'),

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters_adj',
  region = "chr3-90500000-90700000"
)
# S100a6: chr3:90,612,882-90,614,414
# S100a8: chr3:90,668,978-90,670,035
# S100a9: chr3:90,692,632-90,695,721

# S100a6:  chr3:90,612,882-90,614,414
# S100a8: chr3:90,668,978-90,670,035
# S100a9:chr3:90,692,632-90,695,721

# chr3-90500000-90700000 


In [ ]:
options(repr.plot.width = 9, repr.plot.height = 4, repr.plot.res = 200)
CoveragePlot(
  object = oo_msc_5,
  group.by = 'seurat_clusters_adj',
  region = "chr3-90500000-90700000"
)
# S100a6: chr3:90,612,882-90,614,414
# S100a8: chr3:90,668,978-90,670,035
# S100a9: chr3:90,692,632-90,695,721

# S100a6:  chr3:90,612,882-90,614,414
# S100a8: chr3:90,668,978-90,670,035
# S100a9:chr3:90,692,632-90,695,721

# chr3-90500000-90700000

# peak annotate & violin plot for gene expression

In [ ]:
library(eulerr)
library(TxDb.Mmusculus.UCSC.mm10.knownGene)
library(ChIPseeker)


In [ ]:
head(oo_msc_5_peaks)

In [ ]:
ann = annotatePeak(oo_msc_5_peaks, TxDb = TxDb.Mmusculus.UCSC.mm10.knownGene, annoDb = 'org.Mm.eg.db')
ann_kl4 = annotatePeak(oo_msc_5_peaks_4clusters, TxDb = TxDb.Mmusculus.UCSC.mm10.knownGene, annoDb = 'org.Mm.eg.db')


In [ ]:
options(width = 179)

In [ ]:
ann

In [ ]:
plotAnnoBar(ann)

In [ ]:
plotAnnoBar(ann_kl4)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8, repr.plot.res = 300)
plotAnnoPie(ann)

In [ ]:
plotDistToTSS(ann)

In [ ]:
# display the gene expression difference of these genes with chromosome status changes
peak_dt = as.data.table(ann@anno)
peak_dt[, peak_called_in := paste(sort(unlist(strsplit(peak_called_in, ','))), collapse = ','), by = 1:nrow(peak_dt)]
plotdat = peak_dt[peak_called_in %in% c('6') & abs(distanceToTSS) < 5000, .(seqnames, start, end, peak_called_in, annotation, SYMBOL, GENENAME)]
feat_sel = intersect(plotdat$SYMBOL, rna_seu_5_markers[cluster == '6', gene])
feat_sel
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
VlnPlot(object = rna_seu_5, features = feat_sel, ncol = 4, group.by = 'seurat_clusters_adj')


In [ ]:
peak_dt_kl4 = as.data.table(ann_kl4@anno)
peak_dt_kl4[, peak_called_in := paste(sort(unlist(strsplit(peak_called_in, ','))), collapse = ','), by = 1:nrow(peak_dt_kl4)]


In [ ]:
peak_dt_kl4 = as.data.table(ann_kl4@anno)
peak_dt_kl4[, peak_called_in := paste(sort(unlist(strsplit(peak_called_in, ','))), collapse = ','), by = 1:nrow(peak_dt_kl4)]
plotdat = peak_dt_kl4[peak_called_in %in% c('cl_6') & abs(distanceToTSS) < 5000, .(seqnames, start, end, peak_called_in, annotation, SYMBOL, GENENAME)]
feat_sel = intersect(plotdat$SYMBOL, rna_seu_5_markers[cluster == '6', gene])
feat_sel
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
VlnPlot(object = rna_seu_5, features = feat_sel, ncol = 4, group.by = 'seurat_clusters_adj')


In [ ]:
options(repr.plot.width = 9, repr.plot.height = 4, repr.plot.res = 200)
CoveragePlot(
    object = oo_msc_5,
    group.by = 'seurat_clusters_adj',
    region = "chr1-171019764-171019963",
    extend.upstream   = 5000,
    extend.downstream = 5000

)
#chr1	78195219	78195418	6	Promoter (1-2kb)	Pax3	paired box 3
#chr1	80710833	80711059	6	Promoter (1-2kb)	Dock10	dedicator of cytokinesis 10
#chr1	171019764	171019963	6	Promoter (<=1kb)	Fcgr4        	
# lrp1b

# overlapped of peaks from these ATACSeq cell clusters


In [ ]:
peak_dt = as.data.table(ann@anno)
peak_dt[grep('1', peak_called_in), cl1 := 'Yes']
peak_dt[grep('2', peak_called_in), cl2 := 'Yes']
peak_dt[grep('3', peak_called_in), cl3 := 'Yes']
peak_dt[grep('4', peak_called_in), cl4 := 'Yes']

peak_dt[, icl1 := 0] 
peak_dt[, icl2 := 0] 
peak_dt[, icl3 := 0] 
peak_dt[, icl4 := 0] 

peak_dt[grep('1', peak_called_in), icl1 := 1] 
peak_dt[grep('2', peak_called_in), icl2 := 1]
peak_dt[grep('3', peak_called_in), icl3 := 1]
peak_dt[grep('4', peak_called_in), icl4 := 1]
head( peak_dt )


In [ ]:
peak_dt[, nn := icl1 + icl2 + icl3 + icl4 ]
peak_dt[, tag := paste0(seqnames, '_', start, '_', end, '_', width)]

In [ ]:
s4 <- list(
    cl1 = peak_dt[icl1 == 1, tag],
    cl2 = peak_dt[icl2 == 1, tag],
    cl3 = peak_dt[icl3 == 1, tag],
    cl4 = peak_dt[icl4 == 1, tag])
plot(venn(s4))
plot(euler(s4, shape = "ellipse"), quantities = TRUE)


# gene expression of interested genes


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
#VlnPlot(object = rna_seu_5, features = c('S100a1', 'S100a13', 'S100a14', 'S100a16', 'S100a3', 'S100a4', 'S100a5', 'S100a6', 'S100a7', 'S100a8', 'S100a9'), ncol = 5)
VlnPlot(object = rna_seu_5, features = c('S100a1', 'S100a13', 'S100a16', 'S100a4', 'S100a6', 'S100a7', 'S100a8', 'S100a9'), ncol = 4, group.by = 'seurat_clusters_adj')


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
#VlnPlot(object = rna_seu_5, features = c('S100a1', 'S100a13', 'S100a14', 'S100a16', 'S100a3', 'S100a4', 'S100a5', 'S100a6', 'S100a7', 'S100a8', 'S100a9'), ncol = 5)
VlnPlot(object = rna_seu_5, features = c('Smad1',  'Smad2',  'Smad3',  'Smad4',  'Smad5',  'Smad6', 
                                         'Smad7',  'Smad8', 'TGFBR1', 'TGFBR2'), ncol = 4, group.by = 'seurat_clusters_adj')


# FindAllMarkers

In [ ]:
rna_seu_5$seurat_clusters_adj2 = rna_seu_5$seurat_clusters_adj
rna_seu_5$seurat_clusters_adj2[rna_seu_5$seurat_clusters_adj == 5 & rna_seu_5$umap2 < 1] = 4


In [ ]:
options(repr.plot.width = 9, repr.plot.height = 4, repr.plot.res = 200)
DimPlot(rna_seu_5, group.by = "seurat_clusters_adj", label = TRUE) + NoLegend() + ggtitle("RNA")  |
DimPlot(rna_seu_5, group.by = "seurat_clusters_adj2", label = TRUE) + NoLegend() + ggtitle("RNA") 


In [ ]:
table(rna_seu_5$seurat_clusters_adj) 

In [ ]:
Idents(rna_seu_5) = rna_seu_5$seurat_clusters_adj2

In [ ]:
# saved var
rna_seu_5_markers = FindAllMarkers(rna_seu_5, only.pos = T)
rna_seu_5_markers = as.data.table(rna_seu_5_markers, keep.rownames = T)


In [ ]:
rna_seu_5_markers[gene == 'S100a6', ]

In [ ]:
rna_seu_5_markers %>% group_by(cluster) %>% head(10)

In [ ]:
feat_sel = unique(c(
    rna_seu_5_markers[cluster %in% c('1') ][order(p_val_adj), gene][1:5],
    rna_seu_5_markers[cluster %in% c('2') ][order(p_val_adj), gene][1:5],
    rna_seu_5_markers[cluster %in% c('3') ][order(p_val_adj), gene][1:5],
    rna_seu_5_markers[cluster %in% c('4') ][order(p_val_adj), gene][1:5],
    rna_seu_5_markers[cluster %in% c('5') ][order(p_val_adj), gene][1:5],
    rna_seu_5_markers[cluster %in% c('6') ][order(p_val_adj), gene][1:5],
    'S100a6',
    rna_seu_5_markers[cluster %in% c('7') ][order(p_val_adj), gene][1:50]) )
head(feat_sel)


In [ ]:
rna_seu_5_copy

In [ ]:
dim(rna_seu_5_copy@assays$RNA$scale.data)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8, repr.plot.res = 200)
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel , assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
head(rna_seu_5_markers)

In [ ]:
rna_seu_5_copy = copy(rna_seu_5)
Idents(rna_seu_5_copy) = rna_seu_5$kl
rna_seu_5_markers_kl34 = FindAllMarkers(rna_seu_5_copy, only.pos = T)
rna_seu_5_markers_kl34 = as.data.table(rna_seu_5_markers_kl34, keep.rownames = T)
head(rna_seu_5_markers_kl34)


# MS5 cell bulk rna seq

In [ ]:
# to validate these results, a bulk ms5 (mouse MSC cell) bulk RNA-Seq was introduced


In [ ]:
ms5_deg = fread('~/WenhuoHu/collab_JinWang/ms5/deg.xls')
head(ms5_deg) 

In [ ]:
fwrite(rna_seu_5_markers, file = '~/WenhuoHu/collab_JinWang/rna_seu_5_markers.tsv', sep = '\t')

In [ ]:
head(rna_seu_5_markers)

In [ ]:
library(ggVennDiagram)

In [ ]:
gene_sel1 = unique(rna_seu_5_markers[p_val_adj < 0.05, gene]) 
gene_sel2 = unique(ms5_deg[FDR < 0.05, rn])

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 200)
ggVennDiagram(list(gene_sel1, gene_sel2)) + coord_flip()


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8, repr.plot.res = 200)
gene_sel1 = unique(rna_seu_5_markers[p_val_adj < 0.05 & abs(avg_log2FC) > 1, gene]) 
gene_sel2 = unique(ms5_deg[FDR < 0.05 & abs(logFC) > 1, rn])
feat_sel = c(intersect(gene_sel1, gene_sel2), 'S100a6')
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel , assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj2', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
logcpm = read.table('ms5/logcpm_ms5.tsv') 
ms5_dsn = read.table('ms5/dsn.tsv') 

In [ ]:
feat_sel = c(intersect(gene_sel1, gene_sel2), 'S100a6')


In [ ]:
plotdat = logcpm[feat_sel, ms5_dsn$sample_name] 
plotdat = scale_fun(plotdat)
plotdat[plotdat > 2] = 2
plotdat[plotdat < -2] = -2
dim(plotdat)

column_ha = HeatmapAnnotation(df = ms5_dsn[, 'group', drop = F], show_annotation_name = F)

options(repr.plot.width = 4, repr.plot.height = 5, repr.plot.res = 300)
Heatmap(plotdat, show_column_names = F, show_row_names = T, show_column_dend = F, cluster_columns  = F, 
        cluster_rows = T,   row_names_gp = grid::gpar(fontsize = 4),   ##row_split = factor(dsn_row$target),
        show_row_dend = F, top_annotation = column_ha, column_split = ms5_dsn$group) 


In [ ]:
meta_gene = read_xlsx('~/Mammalian_Metabolic_Final.xlsx')
setnames(meta_gene, 1, 'gene')
head(meta_gene) 

In [ ]:
gene_sel1 = unique(rna_seu_5_markers[p_val_adj < 0.05 & abs(avg_log2FC) > 0 & cluster %in% c(5,6,7), gene]) 
gene_sel2 = unique(ms5_deg[FDR < 0.05 & abs(logFC) > 0, rn])
intersect(meta_gene$gene, gene_sel1)
intersect(meta_gene$gene, gene_sel2)
intersect(intersect(meta_gene$gene, gene_sel2), intersect(meta_gene$gene, gene_sel1)) 
feat_sel = intersect(intersect(meta_gene$gene, gene_sel2), intersect(meta_gene$gene, gene_sel1)) 

In [ ]:
plotdat = logcpm[feat_sel, ms5_dsn$sample_name] 
plotdat = scale_fun(plotdat)
plotdat[plotdat > 2] = 2
plotdat[plotdat < -2] = -2
dim(plotdat)

column_ha = HeatmapAnnotation(df = ms5_dsn[, 'group', drop = F], show_annotation_name = F)

options(repr.plot.width = 4, repr.plot.height = 3, repr.plot.res = 300)
Heatmap(plotdat, show_column_names = F, show_row_names = T, show_column_dend = F, cluster_columns  = F, 
        cluster_rows = T,   row_names_gp = grid::gpar(fontsize = 6),   ##row_split = factor(dsn_row$target),
        show_row_dend = F, top_annotation = column_ha, column_split = ms5_dsn$group) 


In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5, repr.plot.res = 200)
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel , assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj2', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
ms5_deg[rn %in% c('Aoc3'), ]

# cell cycle scoring

In [ ]:
# conclustion: no obvious cell cycle difference of these MSC clusters

In [ ]:
geneId = as.data.table(read.csv('~/WenhuoHu/repos/niche/data/human_to_mouse.csv'))
setkey(geneId, 'Gene.name')
head(geneId)

In [ ]:
m.s.genes = geneId[cc.genes$s.genes, Mouse.gene.name]
m.g2m.genes = geneId[cc.genes$g2m.genes, Mouse.gene.name]
 

In [ ]:
m.s.genes = m.s.genes[!is.na(m.s.genes) ]
m.g2m.genes = m.g2m.genes[!is.na(m.g2m.genes) ]
m.s.genes = m.s.genes[m.s.genes != '' ]
m.g2m.genes = m.g2m.genes[m.g2m.genes != '' ]
m.s.genes
m.g2m.genes


In [ ]:
DefaultAssay(rna_seu_5) = 'RNA'


In [ ]:
rna_seu_5 <- CellCycleScoring(rna_seu_5, s.features = m.s.genes, g2m.features = m.g2m.genes, set.ident = TRUE, group.by = 'seurat_clusters_adj')

# view cell cycle scores and phase assignments
head(rna_seu_5@meta.data)

In [ ]:
DimPlot(rna_seu_5, group.by = 'Phase')

In [ ]:
plotdat = as.data.table(rna_seu_5@meta.data)
plotdat = as.data.table(table(plotdat$Phase, plotdat$seurat_clusters_adj)) 
plotdat[, cluster_nn := sum(.SD$N), by = 'V2']
plotdat[, percentage := N / cluster_nn ]
setnames(plotdat, 'V1', 'Phase')
head(plotdat)
ggbarplot(plotdat, x = 'V2', y = 'percentage', fill = 'Phase') + xlab('Clusters') + ylab('Ratio in each cluster')


# genes related with osteoblast cell functions

## David Scaden's results (deprecated)

In [ ]:
# raw data of GSE128432
# /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/scRNA_seq/GSE128432/

In [ ]:
getwd()

In [ ]:
setwd('/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/scRNA_seq/GSE128432/')

In [ ]:
# https://github.com/pblaney/scXploringTheNiche/blob/main/scRnaTaxonomyOfBmStromaHomeostasis.Rmd

In [ ]:
std1 <- Read10X(data.dir = "std1/")
std2 <- Read10X(data.dir = "std2/")
std3 <- Read10X(data.dir = "std3/")
std4 <- Read10X(data.dir = "std4/")
std5 <- Read10X(data.dir = "std5/")
std6 <- Read10X(data.dir = "std6/")

In [ ]:
std1 <- CreateSeuratObject(counts = std1, project = "std1", assay = "RNA")
std2 <- CreateSeuratObject(counts = std2, project = "std2", assay = "RNA")
std3 <- CreateSeuratObject(counts = std3, project = "std3", assay = "RNA")
std4 <- CreateSeuratObject(counts = std4, project = "std4", assay = "RNA")
std5 <- CreateSeuratObject(counts = std5, project = "std5", assay = "RNA")
std6 <- CreateSeuratObject(counts = std6, project = "std6", assay = "RNA")


In [ ]:
std_oo = list(std1, std2, std3, std4, std5, std6)
std_oo = lapply(std_oo, function(oo){
    oo[["Percent_MT"]] <- PercentageFeatureSet(object = oo, pattern = "^mt-")
    oo
    #subset(x = oo, subset = nFeature_RNA < 5000 & nFeature_RNA > 500 & nCount_RNA < 32500 & Percent_MT < 12)
})

In [ ]:
std_oo[[1]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 5000 & nFeature_RNA > 500 & nCount_RNA < 32500 & Percent_MT < 12)
std_oo[[2]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 3250 & nFeature_RNA > 500 & nCount_RNA < 15000 & Percent_MT < 10)
std_oo[[3]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 3000 & nFeature_RNA > 500 & nCount_RNA < 12500 & Percent_MT < 10)
std_oo[[4]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 4250 & nFeature_RNA > 500 & nCount_RNA < 20000 & Percent_MT < 10)
std_oo[[5]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 4500 & nFeature_RNA > 500 & nCount_RNA < 30000 & Percent_MT < 10)
std_oo[[6]] = subset(x = std_oo[[1]], subset = nFeature_RNA < 4000 & nFeature_RNA > 500 & nCount_RNA < 18000 & Percent_MT < 10)


In [ ]:
std_oo <- lapply(X = std_oo, FUN =  function(x) {
    x <- NormalizeData(object = x,
                     normalization.method = "LogNormalize",
                     scale.factor = 10000,
                     verbose = FALSE)

    x <- FindVariableFeatures(object = x,
                            selection.method = "vst",
                            nfeatures = 5000,
                            verbose = FALSE)
    x
})


In [ ]:
stdIntegrationFeatures <- SelectIntegrationFeatures(object.list = std_oo, nfeatures = 5000)

std_oo <- lapply(X = std_oo, FUN = function(x) {
    x <- ScaleData(object = x, features = stdIntegrationFeatures, verbose = FALSE)

    x <- RunPCA(object = x, features = stdIntegrationFeatures, verbose = FALSE)
    x
})

In [ ]:
stdIntegrationAnchors <- FindIntegrationAnchors(object.list = std_oo, reduction = "rpca", anchor.features = stdIntegrationFeatures)
stdIntegrated <- IntegrateData(anchorset = stdIntegrationAnchors)
stdIntegrated <- ScaleData(object = stdIntegrated, verbose = FALSE)

# Space clean up
rm(std1, std2, std3, std4, std5, std6, stdList)

In [ ]:
stdIntegrated

In [ ]:
allGenes <- rownames(stdIntegrated@assays$RNA$counts)


In [ ]:
length(allGenes)

In [ ]:
# First filter out all mitochondrial genes by finding all included in original gene
# list, then checking the variable feature list to see if any of those genes were included.
# If so, remove them from the variable feature set.
mitochondrialGenes <- grep(pattern = "mt-",
                           x = allGenes,
                           ignore.case = TRUE,
                           value = TRUE
                          )

# Remove the mitochondrial genes from the variable features list
stdIntegrated[['integrated']]@var.features <- setdiff(x = stdIntegrated[['integrated']]@var.features,
                                                      y = mitochondrialGenes
                                                     )

# To remove ribosomal protein genes, read in gene list sourced from the HUGO Gene
# Nomenclature Committee (HGNC) and parse it to isolate gene names
hgncRibosomalProteinGeneList <- read_delim(file = "~/repos/scXploringTheNiche/data/hgncRibosomalProteinGeneList.txt",
                                           col_names = TRUE, 
                                           delim = "\t"
                                          )$`Approved symbol`

# Loop through ribosomal protein gene list and determine if that gene is listed
# in the original full gene list
ribosomalProteinGenes <- character()
for(i in 1:length(hgncRibosomalProteinGeneList)) {
    geneMatch <- grep(
        pattern = paste("^", hgncRibosomalProteinGeneList[i], "$", sep = ""),
        x = allGenes,
        ignore.case = TRUE,
        value = TRUE,
        perl = TRUE
    )
    ribosomalProteinGenes <- c(ribosomalProteinGenes, geneMatch)
}
ribosomalProteinGenes <- unique(ribosomalProteinGenes)

# Remove the ribosomal protein genes from the variable features list
stdIntegrated[['integrated']]@var.features <- setdiff(x = stdIntegrated[['integrated']]@var.features,
                                                      y = ribosomalProteinGenes
                                                     )

# To remove cell cycle genes, read in gene list sourced from KEGG/Broad Institute
keggCellCycleGeneList <- as_vector(
    read.csv("~/repos/scXploringTheNiche/data/keggGeneSet.txt", skip = 2, col.names = 'gene')
)

# Loop through cell cycle gene list and determine if that gene is listed
# in the original full gene list
cellCycleGenes <- character()
for(i in 1:length(keggCellCycleGeneList)) {
    geneMatch <- grep(
        pattern = paste("^", keggCellCycleGeneList[i], "$", sep = ""),
        x = allGenes,
        ignore.case = TRUE,
        value = TRUE,
        perl = TRUE
    )
    cellCycleGenes <- c(cellCycleGenes, geneMatch)
}

# Remove the cell cycle genes from the variable features list
stdIntegrated[['integrated']]@var.features <- setdiff(x = stdIntegrated[['integrated']]@var.features,
                                                      y = cellCycleGenes
                                                     )

In [ ]:
stdIntegrated <- RunPCA(object = stdIntegrated, verbose = FALSE, npcs = 65)

# Produce a PCA plot where each point represents a cell and its position relative to the
# origin point (0,0) shows how much influence it has on the that particular PC. Additionally,
# the correlation between two points can be determined by the angle between the vectors from
# each point to the origin point. Smaller the angle, the more positive correlation.
# Perpendicular angles mean likely no correlation.
DimPlot(object = stdIntegrated, reduction = "pca")

# Display a heatmap showing how genes correlate with what PC. Each line represents a cell
# and the color represents the loading of the gene (the genes shown are most extremely variable)
# with black being 0, yellow being positive and purple being negative.
DimHeatmap(object = stdIntegrated, dims = 1, balanced = TRUE)

In [ ]:
ElbowPlot( object = stdIntegrated,  ndims = 65) + geom_abline( aes(intercept = 1.75, slope = 0, color = "red"), show.legend = FALSE)

In [ ]:
stdIntegrated

In [ ]:
names(stdIntegrated@graphs)

In [ ]:
stdIntegrated <- FindNeighbors(object = stdIntegrated, dims = 1:50)
stdIntegrated <- FindClusters(object = stdIntegrated, resolution = 0.64, random.seed = 2019, reduction = 'pca')

In [ ]:
stdIntegrated <- RunTSNE(object = stdIntegrated, dims = 1:50)

# Plot the tSNE reduction grouped by sample origin then by cluster
DimPlot(object = stdIntegrated, reduction = "tsne", group.by = 'orig.ident')

DimPlot(object = stdIntegrated, reduction = "tsne", label = TRUE, label.size = 5,
        cols = c("aquamarine2", "blueviolet", "chartreuse2", "darkgreen", "darkred", 
                 "darkgray", "chocolate1", "coral4", "blue", "darkmagenta", "deeppink2",
                 "goldenrod1", "yellow1", "tan4", "tomato1", "olivedrab", "navajowhite2",
                 "orangered", "palegreen", "peru", "plum1", "purple3", "red2", "salmon",
                 "seagreen", "magenta", "midnightblue", "khaki", "greenyellow", "deeppink",
                 "burlywood3", "dimgray"))


In [ ]:
saveRDS(object = stdIntegrated, file = "data/stdIntegrated.rds")
#stdIntegrated <- readRDS(file = "data/stdIntegrated.rds")

In [ ]:
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Cxcl12", "Bglap", "Cdh5", "Acan", "S100a4", "Acta2"),
            reduction = "tsne",
            label = TRUE,
            ncol = 3,
            cols = c("lightskyblue1", "red3"))

# Plot the distribution of expression for all cells related to specific genes
# with a violin plot
VlnPlot(object = stdIntegrated,
        features = c("Cxcl12", "Bglap", "Cdh5", "Acan", "S100a4", "Acta2"),
        pt.size = 0.5,
        ncol = 2)
DefaultAssay(stdIntegrated) <- "integrated"

In [ ]:
# Visualize the distribution of cells expressing gene markers across all clusters
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Cd79a", "Cd79b"),
            reduction = "tsne",
            label = TRUE,
            label.size = 4,
            cols = c("lightskyblue1", "red3"))
# Appears to be 7, 9, 12, 25, 27 
VlnPlot(object = stdIntegrated,
        features = c("Cd79a", "Cd79b"),
        pt.size = 0.5, 
        idents = c(7, 9, 12, 25, 27, 1, 2))

### NOTE ###
# These methods are loose and not rooted in any statistical rule, depreceated
# Determine cutoff values for expression levels of gene markers based on expression across all cells
#markerCd79a <- stdIntegrated[["integrated"]]@scale.data["Cd79a",]
#quantile(markerCd79a, probs = seq(0.6, 1, 0.025))

#markerCd79b <- stdIntegrated[["integrated"]]@scale.data["Cd79b",]
#quantile(markerCd79b, probs = seq(0.6, 1, 0.025))

# Identify all the cells that satisfy cutoff value which was selected to be between 85 and 90 quantile
#clusterFilter1 <- WhichCells(object = stdIntegrated,
#                             expression = Cd79a > 1 & Cd79b > 1)

# Visualize count of cells per cluster
#clusterFilter1 <- table(stdIntegrated$seurat_clusters[clusterFilter1])

bcellClusters <- c(7, 9, 12, 25, 27)
DefaultAssay(stdIntegrated) <- "integrated"

In [ ]:
# Visualize the distribution of cells expressing gene markers across all clusters
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Gypa", "Hbb-bt", "Hbb-bs", "Rhag", "Rhd", "Tfrc"),
            reduction = "tsne",
            label = TRUE,
            label.size = 4,
            cols = c("lightskyblue1", "red3"))
# looks like 5, 18
VlnPlot(object = stdIntegrated,
        features = c("Gypa", "Hbb-bt", "Hbb-bs", "Rhag", "Rhd", "Tfrc"),
        pt.size = 0.5, 
        idents = c(5, 18, 0, 1, 2))

# Determine cutoff values for expression levels of gene markers based on expression across all cells
#markerGypa <- stdIntegrated[["integrated"]]@scale.data["Gypa",]
#quantile(markerGypa, probs = seq(0.6, 1, 0.025))

#markerHbbbt <- stdIntegrated[["integrated"]]@scale.data["Hbb-bt",]
#quantile(markerHbbbt, probs = seq(0.6, 1, 0.025))

#markerHbbbs <- stdIntegrated[["integrated"]]@scale.data["Hbb-bs",]
#quantile(markerHbbbs, probs = seq(0.6, 1, 0.025))

#markerRhag <- stdIntegrated[["integrated"]]@scale.data["Rhag",]
#quantile(markerRhag, probs = seq(0.6, 1, 0.025))

#markerRhd <- stdIntegrated[["integrated"]]@scale.data["Rhd",]
#quantile(markerRhd, probs = seq(0.6, 1, 0.025))

#markerTfrc <- stdIntegrated[["integrated"]]@scale.data["Tfrc",]
#quantile(markerTfrc, probs = seq(0.6, 1, 0.025))

# Identify all the cells that satisfy cutoff value which was selected based on distribution
#clusterFilter2 <- WhichCells(
#        object = stdIntegrated,
#        expression = Gypa > 0.15 & "Hbb-bt" > 0.15 & "Hbb-bs" > 0.15 & Rhag > 0.15 & Rhd > 0.15 & Tfrc > 0.15)

# Visualize count of cells per cluster
#clusterFilter2 <- table(stdIntegrated$seurat_clusters[clusterFilter2])


erythroidClusters <- c(5,18)
DefaultAssay(stdIntegrated) <- "integrated"

In [ ]:
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Cd52", "Cd177", "Plaur", "Clec4a2"),
            reduction = "tsne",
            label = TRUE,
            label.size = 4,
            cols = c("lightskyblue1", "red3"))
# looks like 14, 16, 31
VlnPlot(object = stdIntegrated,
        features = c("Cd52", "Cd177", "Plaur", "Clec4a2"),
        pt.size = 0.5, 
        idents = c(14, 16, 31, 0, 1, 2),
        ncol = 2)

# Determine cutoff values for expression levels of gene markers based on expression across all cells
#markerCd52 <- stdIntegrated[["integrated"]]@scale.data["Cd52",]
#quantile(markerCd52, probs = seq(0.6, 1, 0.025))

#markerCd177 <- stdIntegrated[["integrated"]]@scale.data["Cd177",]
#quantile(markerCd177, probs = seq(0.6, 1, 0.025))

#markerPlaur <- stdIntegrated[["integrated"]]@scale.data["Plaur",]
#quantile(markerPlaur, probs = seq(0.6, 1, 0.025))

#markerClec4a2 <- stdIntegrated[["integrated"]]@scale.data["Clec4a2",]
#quantile(markerClec4a2, probs = seq(0.6, 1, 0.025))

# Identify all the cells that satisfy cutoff value which was selected based on distribution
#clusterFilter3 <- WhichCells(object = stdIntegrated,
#                             expression = Cd52 > 0.4 & Cd177 > 0.075 & Plaur > 0.25 & Clec4a2 > 0.5)

# Visualize count of cells per cluster
#clusterFilter3 <-table(stdIntegrated$seurat_clusters[clusterFilter3])

granulocyteClusters <- c(14, 16, 31)
DefaultAssay(stdIntegrated) <- "integrated"


In [ ]:
# Visualize the distribution of cells expressing gene markers across all clusters
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Gp9", "Itga2b", "Cd9", "Gp1bb"),
            reduction = "tsne",
            label = TRUE,
            label.size = 4,
            cols = c("lightskyblue1", "red3"))
# looks like 21, 28
VlnPlot(object = stdIntegrated,
        features = c("Gp9", "Itga2b", "Cd9", "Gp1bb"),
        pt.size = 0.5,
        idents = c(21, 28, 0, 1, 2),
        ncol = 2)

# Determine cutoff values for expression levels of gene markers based on expression across all cells
#markerGp9 <- stdIntegrated[["integrated"]]@scale.data["Gp9",]
#quantile(markerGp9, probs = seq(0.6, 1, 0.025))

#markerItga2b <- stdIntegrated[["integrated"]]@scale.data["Itga2b",]
#quantile(markerItga2b, probs = seq(0.6, 1, 0.025))

#markerCd9 <- stdIntegrated[["integrated"]]@scale.data["Cd9",]
#quantile(markerCd9, probs = seq(0.6, 1, 0.025))

#markerGp1bb <- stdIntegrated[["integrated"]]@scale.data["Gp1bb",]
#quantile(markerGp1bb, probs = seq(0.6, 1, 0.025))

# Identify all the cells that satisfy cutoff value which was selected based on distribution
#clusterFilter5 <- WhichCells(object = stdIntegrated,
#                             expression = Gp9 > 0.75 & Itga2b > 0.75 & Cd9 > 1.25 & Gp1bb > 1)

# Visualize count of cells per cluster
#clusterFilter5 <-table(stdIntegrated$seurat_clusters[clusterFilter5])

macroAndMegaClusters <- c(21, 28)
DefaultAssay(stdIntegrated) <- "integrated"

In [ ]:
# Visualize the distribution of cells expressing gene markers across all clusters
DefaultAssay(stdIntegrated) <- "RNA"
FeaturePlot(object = stdIntegrated,
            features = c("Ms4a3", "Clec12a", "Fcgr3"),
            reduction = "tsne",
            label = TRUE,
            label.size = 4,
            cols = c("lightskyblue1", "red3"))
# looks like maybe 26
VlnPlot(object = stdIntegrated,
        features = c("Ms4a3", "Clec12a", "Fcgr3"),
        pt.size = 0.5,
        idents = c(26, 0, 1, 2),
        ncol = 2)

# Determine cutoff values for expression levels of gene markers based on expression across all cells
#markerMs4a3 <- stdIntegrated[["integrated"]]@scale.data["Ms4a3",]
#quantile(markerMs4a3, probs = seq(0.6, 1, 0.025))

#markerClec12a <- stdIntegrated[["integrated"]]@scale.data["Clec12a",]
#quantile(markerClec12a, probs = seq(0.6, 1, 0.025))

#markerFcgr3 <- stdIntegrated[["integrated"]]@scale.data["Fcgr3",]
#quantile(markerFcgr3, probs = seq(0.6, 1, 0.025))

# Identify all the cells that satisfy cutoff value which was selected based on distribution
#clusterFilter6 <- WhichCells(object = stdIntegrated,
#                             expression = Ms4a3 > 0.1 & Clec12a > 0.33 & Fcgr3 > 0.5)

# Visualize count of cells per cluster
#clusterFilter6 <-table(stdIntegrated$seurat_clusters[clusterFilter6])

additionalContaminationCluster <- 26
DefaultAssay(stdIntegrated) <- "integrated"

In [ ]:
bcellClusters <- c(7, 9, 12, 25, 27)


In [ ]:
hematopoieticClusters <- c(bcellClusters,
                           erythroidClusters,
                           granulocyteClusters,
                           macroAndMegaClusters,
                           additionalContaminationCluster)

# List all clusters that were annotated as non-hematopoietic
nonHematopoieticClusters <- as.numeric(levels(stdIntegrated@active.ident)) %>% setdiff(y = hematopoieticClusters)

# Total all the cells that were caught by filters per each cluster
hematopoieticFilterCount <- c()
for(i in 1:length(hematopoieticClusters)) {
  hematopoieticFilterCount[i] <- CellsByIdentities(object = stdIntegrated,
                                                   idents = hematopoieticClusters[i]) %>%
                                   as_vector() %>%
                                   length()
}
sum(hematopoieticFilterCount)

In [ ]:
stdStromal <- subset(x = stdIntegrated,  idents = nonHematopoieticClusters)

# Rerun the PCA on the stromal population clusters alone
stdStromal <- RunPCA(object = stdStromal, npcs = 50)

# Recalculate the neighbors and clusters for the stromal population 
stdStromal <- FindNeighbors(object = stdStromal, dims = 1:50)

stdStromal <- FindClusters(object = stdStromal, resolution = 0.38, random.seed = 2019)

# Rerun the tSNE on the stromal population
stdStromal <- RunTSNE(object = stdStromal, dims = 1:50)



In [ ]:
# Plot the new tSNE of stromal population
options(repr.plot.width = 7, repr.plot.height = 5, repr.plot.res = 300)
DimPlot( object = stdStromal, reduction = "tsne", label = TRUE, label.size = 5) 


In [ ]:
saveRDS(object = stdStromal, file = "stdStromal.rds")
#stdStromal <- readRDS(file = "data/stdStromal.rds")

In [ ]:
# Plot keystone gene markers for all 6 major populations
options(repr.plot.width = 17, repr.plot.height = 10, repr.plot.res = 300)
DefaultAssay(stdStromal) <- "RNA"
FeaturePlot(object = stdStromal,
            features = c("Cxcl12", "Bglap", "Cdh5", "Acan", "S100a4", "Acta2"),
            reduction = "tsne", label = TRUE, label.size = 5, ncol = 3, cols = c("lightskyblue1", "red3"))
DefaultAssay(stdStromal) <- "integrated"

In [ ]:
atlas = as.data.table(read_excel('~/NIHMS1529101-supplement-8.xlsx', skip = 1))
head(atlas)

## marker genes

In [ ]:
# Markers for each population as per Scadden et al.
marker_MSC = c("Lepr", "Adipoq", "Cxcl12", "Kitl", "Grem1", "Vcam1")
marker_OLC = c("Bglap", "Runx2", "Sp7", "Cd200", "Spp1", "Grem1", "Mmp13")
marker_Chondro = c("Sox9", "Acan", "Col2a1", "Ihh", "Pth1r", "Mef2c")
marker_Pericyte = c("Acta2", "Nes", "Cspg4", "Myh11", "Mcam")
marker_EC = c("Pecam1", "Cdh5", "Cd34", "Kdr", "Emcn", "Flt4", "Ly6a")
marker_Fibro = c("Fn1", "S100a4", "Dcn", "Sema3c", "Cd34", "Ly6a", "Pdgfra", "Thy1", "Cd44", "Sox9", "Scx", "Spp1", "Nt5e", "Cspg4", "Cilp")
marker_LeprMSC = c("Lepr", "Adipoq", "Cxcl12", "Kitl", "Grem1", "Vcam1")
marker_OLC1 = c("Bglap", "Runx2", "Sp7", "Cxcl12 high", "Cd200", "Spp1")
marker_OLC2 = c("Bglap", "Runx2", "Sp7", "Cxcl12", "Grem1", "Mmp13")
marker_Chondrohyper = c("Sox9", "Acan", "Col10a1", "Ihh", "Runx2", "Mef2c")
marker_Chondroprogen = c("Sox9", "Acan", "Col2a1", "Grem1", "Runx2", "Sp7", "Alpl", "Spp1")
marker_Chondro = c("Sox9", "Acan", "Col2a1")
marker_Chondroprehyper2 = c("Sox9", "Acan", "Col2a1", "Ihh", "Pth1r", "Mef2c")
marker_ChondroprolRest = c("Sox9", "Acan", "Col2a1")
marker_Pericytes = c("Acta2", "Nes", "Cspg4", "Myh11", "Mcam")
marker_ECsinusoidal = c("Flt4", "Ly6a", "Cd34", "Il6st")
marker_ECarteriolar = c("Flt4", "Ly6a", "Cd34", "Il6st")
marker_ECarterial = c("Flt4", "Ly6a", "Cd34", "Il6st", "Kitl")
marker_Fibro = c("Fn1", "S100a4", "Dcn", "Sema3c", "Cd34", "Ly6a", "Pdgfra",  "Thy1", "Cd44", "Sox9", "Scx", "Spp1", "Nt5e", "Cspg4", "Cilp")
marker_LeprMSC = c("Lepr", "Adipoq", "Cxcl12", "Kitl", "Grem1", "Vcam1")

In [ ]:
all_markers = unique(c("Lepr", "Adipoq", "Cxcl12", "Kitl", "Grem1", "Vcam1", "Bglap", "Runx2", "Sp7", "Cd200", "Spp1", "Grem1", "Mmp13", "Sox9", "Acan", "Col2a1", "Ihh", "Pth1r", "Mef2c", "Acta2", "Nes", "Cspg4", "Myh11", "Mcam",
  "Pecam1", "Cdh5", "Cd34", "Kdr", "Emcn", "Flt4", "Ly6a", "Lepr", "Adipoq", "Cxcl12", "Kitl", "Grem1", "Vcam1", "Bglap", "Runx2", "Sp7", "Cxcl12", "Cd200", "Spp1",
  "Bglap", "Runx2", "Sp7", "Cxcl12", "Grem1", "Mmp13", "Sox9", "Acan", "Col10a1", "Ihh", "Runx2", "Mef2c", "Sox9", "Acan", "Col2a1", "Grem1", "Runx2", "Sp7", "Alpl", "Spp1",
  "Sox9", "Acan", "Col2a1", "Sox9", "Acan", "Col2a1", "Ihh", "Pth1r", "Mef2c", "Sox9", "Acan", "Col2a1",
  "Acta2", "Nes", "Cspg4", "Myh11", "Mcam", "Flt4", "Ly6a", "Cd34", "Il6st", "Flt4", "Ly6a", "Cd34", "Il6st",
  "Flt4", "Ly6a", "Cd34", "Il6st", "Kitl", "Fn1", "S100a4", "Dcn", "Sema3c", "Cd34", "Ly6a", "Pdgfra",  "Thy1", "Cd44", "Sox9", "Scx", "Spp1", "Nt5e", "Cspg4", "Cilp",
  "Fn1", "S100a4", "Dcn", "Sema3c", "Cd34", "Ly6a", "Pdgfra", "Thy1", "Cd44", "Sox9", "Scx", "Spp1", "Nt5e", "Cspg4", "Cilp"))


In [ ]:
# add interested gene 
all_markers = c(all_markers, 'S100a6', 'S100a8', 'S100a9')


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6, repr.plot.res = 200)
feat_sel = all_markers
feat_sel = c(feat_sel, 'LAT2')
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel, assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 200)
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))

In [ ]:
# '~/Copy of Sabatini Metabolic Gene List (Human and Mouse).xlsx'
# meta = as.data.table(read_excel('~/Mammalian_Metabolic_Final.xlsx'))
# https://esbl.nhlbi.nih.gov/Databases/KSBP2/Targets/Lists/MetabolicEnzymes/MetabolicEnzymeDatabase.html

In [ ]:
meta = fread('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/repos/niche/data/Mouse_metab_genes.txt')

In [ ]:
head(rna_seu_5_markers[gene %in% meta$Symbol, ])
table(rna_seu_5_markers[gene %in% meta$Symbol, cluster])


In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6, repr.plot.res = 200)
#feat_sel = rna_seu_5_markers[gene %in% meta$Symbol,] %>% group_by(cluster) %>% slice_min(p_val_adj, n = 20)
#feat_sel = feat_sel$gene
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel, assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
feat_sel = rna_seu_5_markers[cluster == '6' & gene %in% meta$Symbol & p_val_adj < 0.05 & avg_log2FC > 1, ]
dim(feat_sel)


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 200)
feat_sel = rna_seu_5_markers[cluster == '6' & gene %in% meta$Symbol & p_val_adj < 0.05 & avg_log2FC > 1, ][1:60, gene]
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel, assay = 'RNA') 
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 200)
feat_sel = rna_seu_5_markers[cluster == '6' & gene %in% meta$Symbol & p_val_adj < 0.05 & avg_log2FC > 1, ][61:120, gene]
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel, assay = 'RNA') 
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 200)
feat_sel = rna_seu_5_markers[cluster == '6' & gene %in% meta$Symbol & p_val_adj < 0.05 & avg_log2FC > 1, ][121:171, gene]
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel, assay = 'RNA') 
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 200)
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))

# monocle3 trajectory analysis 

In [ ]:
library(monocle3)
library(SeuratWrappers)


In [ ]:
GM_state <- function(cds){
  if (length(unique(pData(cds)$State)) > 1){
    T0_counts <- table(pData(cds)$State, pData(cds)$Hours)[,"0"]
    return(as.numeric(names(T0_counts)[which
          (T0_counts == max(T0_counts))]))
  } else {
    return (1)
  }
}


In [ ]:
rna_seu_5

In [ ]:
cds <- as.cell_data_set(rna_seu_5, )
#cds <- preprocess_cds(cds, num_dim = 50)
#cds <- reduce_dimension(cds)
cds <- cluster_cells(cds, resolution=1e-3)

p1 <- plot_cells(cds, color_cells_by = "cluster", show_trajectory_graph = FALSE) #, root_state =)
p2 <- plot_cells(cds, color_cells_by = "partition", show_trajectory_graph = FALSE)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)
wrap_plots(p1, p2)


In [ ]:
rowData(cds) = data.frame(gene_short_name = rownames(cds) )


In [ ]:
cds <- learn_graph(cds, use_partition = F, verbose = FALSE)


In [ ]:
saveRDS(cds, file = 'data/msc_monocle3.rds')

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 200)
plot_cells(cds, color_cells_by = "cluster", label_groups_by_cluster=FALSE, label_leaves=FALSE, label_branch_points=FALSE)


In [ ]:
# saved var
cds <- order_cells(cds, root_cells = colnames(cds[,cds$seurat_clusters_adj == 1]))


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 200)
plot_cells(cds, 
           color_cells_by = "pseudotime",
           group_cells_by = "cluster",
           label_cell_groups = FALSE,
           label_groups_by_cluster=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE,
           label_roots = F,
           trajectory_graph_color = "grey60")


In [ ]:
marker_test_res <- top_markers(cds, group_cells_by="seurat_clusters_adj",  cores=8)


In [ ]:
top_specific_markers <- marker_test_res %>% filter(fraction_expressing >= 0.10) %>% group_by(cell_group) %>% top_n(5, pseudo_R2)

top_specific_marker_ids <- unique(top_specific_markers %>% pull(gene_id))


In [ ]:
plot_cells(cds, genes=c("S100a6"), show_trajectory_graph=FALSE, label_cell_groups=FALSE, label_leaves=FALSE)

In [ ]:
ciliated_cds_pr_test_res <- graph_test(cds, neighbor_graph="principal_graph", cores=4)


In [ ]:
pr_deg_ids = as.data.table(ciliated_cds_pr_test_res, keep.rownames = T)
pr_deg_ids = pr_deg_ids[order(q_value, -morans_test_statistic), ]
head(pr_deg_ids)


In [ ]:
pr_deg_ids <- row.names(subset(ciliated_cds_pr_test_res, q_value < 0.001))
length(pr_deg_ids)

In [ ]:
gene_module_df <- find_gene_modules(cds[pr_deg_ids,], resolution=c(10^seq(-6,-1)))


In [ ]:
plot_cells(cds, genes=gene_module_df %>% filter(module %in% c(27, 10, 7, 30)),
           label_cell_groups=FALSE, show_trajectory_graph=FALSE)

In [ ]:
AFD_genes <- c("gcy-8", "dac-1", "oig-8")
AFD_lineage_cds <- cds[rowData(cds)$gene_short_name %in% AFD_genes,
                       colData(cds)$cell.type %in% c("AFD")]
AFD_lineage_cds <- order_cells(AFD_lineage_cds)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)
plot_genes_in_pseudotime(cds[rowData(cds)$gene_short_name %in% c('S100a6', pr_deg_ids[1:5, rn]), ],
                         #color_cells_by="embryo.time.bin",
                         min_expr=0.5)

In [ ]:
library(velocyto.R)
library(SeuratWrappers)


In [ ]:
design[, velo_dir := paste0(getwd(), '/velo/', sample_name, '/')]


In [ ]:
design[, {dir.create(velo_dir)}, by = 1:nrow(design) ]


In [ ]:
design[1, velo_dir]

In [ ]:
mm10_gtf = 'research/groups/jxugrp/home/common/Lab_Members/WenhuoHu/igenomes/Mus_musculus/UCSC/mm10/Annotation/Archives/archive-2015-07-17-14-33-26/Genes/genes.gtf'

In [ ]:
design[, gex_bam_file := sub('atac_', 'gex_', bam_file) ]
design[1, gex_bam_file]


In [ ]:
file.exists(design$gex_bam_file)

In [ ]:
GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/hs1.ncbiRefSeq.gtf"
TE_GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/T2T_CHM13_v2_rmsk_TE.gtf"
bin_tecount = '/home/whu78/conda3/envs/te/bin/'

mm10_gtf = '/research/groups/jxugrp/home/common/Lab_Members/WenhuoHu/igenomes/Mus_musculus/UCSC/mm10/Annotation/Archives/archive-2015-07-17-14-33-26/Genes/genes.gtf'
design[, gex_bam_file := sub('atac_', 'gex_', bam_file) ]

design[, velo_jobname := paste0('velo_', sample_name)]
design[, velo_oo := paste0(velo_dir, 'velo_', sample_name, '_out')]
design[, velo_eo := paste0(velo_dir, 'velo_', sample_name, '_err')]

design[, velo_cmd := paste0('bsub -J ', velo_jobname, ' -n 10 -o ', velo_oo, ' -e ', velo_eo, ' -P velo -R "rusage[mem=5GB]" ')]
design[, velo_cmd := paste0(velo_cmd, ' -W 666:00')]
design[, velo_cmd := paste0(velo_cmd, ' " velocyto run -@ 10 -o ', velo_dir, ' ', design$gex_bam_file, ' ', mm10_gtf, '"')]

write(design[, velo_cmd], file = paste0(base_dir, 'run_velo.sh'))


In [ ]:
tmp = fread('velo/loom_list', header = F)
setnames(tmp, 1, 'loom_file')
tmp[, sample_name := sub("..(.*).gex.*", '\\1', loom_file) ]
tmp[, loom_file := paste0('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_JinWang/velo/', loom_file) ]
tmp

In [ ]:
design = merge(design, tmp, by = 'sample_name')

In [ ]:
ma9_msc_samples = c('M2', 'M3', 'M4', 'M4_m10', 'ctrl_m4w', 'ctrl_m6w', 'ctrl_m8w')
dsn_sel = design[sample_name %in% ma9_msc_samples,] 


In [ ]:
msc_design$sample_name
dsn_sel$sample_name

In [ ]:
mtx_list = lapply(dsn_sel$loom_file, function(fname){ReadVelocity(fname)})

In [ ]:
dsn_sel$sampleID2

In [ ]:
rna_cells = sub('-.*', '', Cells(rna_seu_5))


In [ ]:
head(rna_cells)

In [ ]:
rowname_list = lapply(1:length(mtx_list), function(ii){
    rownames(mtx_list[[ii]][[1]])
})

In [ ]:
gene_sel = Reduce(intersect, rowname_list)
length(gene_sel) 


In [ ]:
mtx_list_2 = lapply(1:length(mtx_list), function(ii){
    mtx = mtx_list[[ii]]
    sname = dsn_sel[ii, sampleID2]
    cell_id = colnames(mtx[[1]])
    cell_id = sub('.*:', '', cell_id)
    cell_id = paste0(sname, '__', cell_id)
    
    mtx1 = mtx[[1]]
    colnames(mtx1) = cell_id
    mtx[[1]] = mtx1[gene_sel, cell_id %in% rna_cells]
    
    mtx2 = mtx[[2]]
    colnames(mtx2) = cell_id
    mtx[[2]] = mtx2[gene_sel, cell_id %in% rna_cells]
    
    mtx3 = mtx[[3]]
    colnames(mtx3) = cell_id
    mtx[[3]] = mtx3[gene_sel, cell_id %in% rna_cells]
    mtx
    
})

In [ ]:
lapply(1:length(mtx_list), function(ii){
    dim(mtx_list[[ii]][[1]])
})

In [ ]:
lapply(1:length(mtx_list_2), function(ii){
    dim(mtx_list_2[[ii]][[1]])
})

In [ ]:
mtx1_list = lapply(mtx_list_2, function(xx){xx[[1]]})
mtx2_list = lapply(mtx_list_2, function(xx){xx[[2]]})
mtx3_list = lapply(mtx_list_2, function(xx){xx[[3]]})

In [ ]:
mtx1 = Reduce(cbind, mtx1_list)
mtx2 = Reduce(cbind, mtx2_list)
mtx3 = Reduce(cbind, mtx3_list)

In [ ]:
# saved var
mtx_sel_list = list(mtx1, mtx2, mtx3)
names(mtx_sel_list) = names(mtx_list[[1]])
names(mtx_sel_list)
dim(mtx_sel_list[[1]])


In [ ]:
tmp = colnames(mtx_sel_list[[1]])
tmp2 = make.unique(tmp, sep = '_')
colnames(mtx_sel_list[[1]]) = tmp2
colnames(mtx_sel_list[[2]]) = tmp2
colnames(mtx_sel_list[[3]]) = tmp2

In [ ]:
names(mtx_sel_list)

In [ ]:
row_sum = rowSums(mtx_sel_list[[1]])
col_sum = colSums(mtx_sel_list[[1]])

In [ ]:
sum(row_sum > 1000)
sum(col_sum > 1000)

In [ ]:
col_sel = col_sum > 1000

In [ ]:
mtx_sel_list_2 = lapply(mtx_sel_list, function(xx){xx[, col_sel]})

In [ ]:
dim(mtx_sel_list_2[[1]])

In [ ]:
velo_seu = as.Seurat(mtx_sel_list)

In [ ]:
save.image(file = 'data/nb_niche_MSC_ATAC_R_Jul18.rdata')


In [ ]:
base::load('data/nb_niche_MSC_ATAC_R_Jul18.rdata')


In [ ]:
velo_seu <- SCTransform(object = velo_seu, assay = "spliced")
velo_seu <- RunPCA(object = velo_seu, verbose = FALSE)
velo_seu <- FindNeighbors(object = velo_seu, dims = 1:20)
velo_seu <- FindClusters(object = velo_seu)
velo_seu <- RunUMAP(object = velo_seu, dims = 1:20)
velo_seu <- RunVelocity(object = velo_seu, deltaT = 1, kCells = 25, fit.quantile = 0.02)
ident.colors <- (scales::hue_pal())(n = length(x = levels(x = velo_seu)))
names(x = ident.colors) <- levels(x = velo_seu)
cell.colors <- ident.colors[Idents(object = velo_seu)]
names(x = cell.colors) <- colnames(x = velo_seu)


In [ ]:
velo_seu

In [ ]:
getwd()

In [ ]:
velo_seu_tool = Tool(object = velo_seu,  slot = "RunVelocity")

In [ ]:
names(velo_seu_tool)

In [ ]:
saveRDS(velo_seu_tool, file = 'data/velo_seu_tool.rds')

In [ ]:
saveRDS(velo_seu, file = 'data/velo_seu.rds')

In [ ]:
show.velocity.on.embedding.cor(emb = Embeddings(object = velo_seu, reduction = "umap"), vel = velo_seu_tool, n = 200, 
                               scale = "sqrt", cell.colors = ac(x = cell.colors, alpha = 0.5), 
                               cex = 0.8, arrow.scale = 3, show.grid.flow = TRUE, min.grid.cell.mass = 0.5, grid.n = 40, arrow.lwd = 1, 
                               do.par = FALSE, cell.border.alpha = 0.1)


# motif analysis

In [ ]:
#library(JASPAR2020)
library(BSgenome.Mmusculus.UCSC.mm10)


In [ ]:
library(Signac)
library(Seurat)
#library(TFBSTools)

In [ ]:
# Get a list of motif position frequency matrices from the JASPAR database
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)


In [ ]:
pfm = readRDS('data/pfm.rds')


In [ ]:
# add motif information
oo_msc_5 <- AddMotifs(
  object = oo_msc_5,
  genome = BSgenome.Mmusculus.UCSC.mm10,
  pfm = pfm
)


In [ ]:
# saved var
oo_msc_5 = FRiP(oo_msc_5, assay = 'ATAC', total.fragments = 'fragments')


In [ ]:
Idents(oo_msc_5) = oo_msc_5$seurat_clusters_adj


In [ ]:
da_peaks <- FindMarkers( object = oo_msc_5, ident.1 = '6',  ident.2 = c('1', '2', '3', '4'),  only.pos = TRUE, test.use = 'LR', min.pct = 0.05, latent.vars = 'nCount_ATAC' )
da_peaks <- rownames(da_peaks[da_peaks$p_val < 0.005 & da_peaks$pct.1 > 0.2, ])
enriched_motifs  <- FindMotifs( object = oo_msc_5, features = da_peaks )


In [ ]:
head(enriched_motifs)


In [ ]:
mm = intersect(toupper(unlist(unname(rna_seu_5_markers[cluster == '4', 'gene']))), enriched_motifs[enriched_motifs$p.adjust < 0.05, 'motif.name'])
mm_hsa = c('CEBPD', 'CEBPE', 'STAT3', 'JUND', 'KLF5', 'FOSL1')
mm_mmu = c('Cebpd', 'Cebpe', 'Stat3', 'Jund', 'Klf5', 'Fosl1')
mm_mmu_2 = c('Cebpa', 'Cebpb', 'Cebpd', 'Cebpe', 'Stat3', 'Jund', 'Klf5', 'Fosl1')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5, repr.plot.res = 300)
motif_sel = c('FOSL2', 'FOS', 'JUND', 'JUNB', 'BATF3', 'BACH2', 'NFE2', 'JDP2')
MotifPlot( object = oo_msc_5, motifs = toupper(motif_sel), ncol = 4)


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4, repr.plot.res = 300)
motif_sel = c('SMAD3', 'RBPJ', 'RUNX1', 'SP1')
MotifPlot( object = oo_msc_5, motifs = toupper(motif_sel), ncol = 4)


In [ ]:
motif_sel = c('FOSL2', 'FOS', 'JUND', 'JUNB', 'BATF3', 'BACH2', 'NFE2', 'JDP2', 'SMAD3', 'RBPJ', 'RUNX1', 'SP1')
motif_sel = c('Fosl2', 'Fos', 'Jund', 'Junb', 'Batf3', 'Bach2', 'Nfe2', 'Jdp2', 'Smad3', 'Rbpj', 'RUNX1', 'Sp1')


In [ ]:
options(repr.plot.width = 8, repr.plot.height = 3, repr.plot.res = 200)
rna_seu_5_copy = ScaleData(rna_seu_5, features = motif_sel, assay = 'RNA') 
DotPlot(rna_seu_5_copy, features = motif_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 3, repr.plot.res = 200)
rna_seu_5_copy = ScaleData(rna_seu_5, features = mm_mmu_2, assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = mm_mmu_2, group.by = 'seurat_clusters_adj', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
VlnPlot(object = rna_seu_5, features = mm_mmu_2, ncol = 4, group.by = 'seurat_clusters_adj')


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)
MotifPlot( object = oo_msc_5, motifs = head(rownames(enriched_motifs), 20) )


# pathway analysis 

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 200)
DimPlot(rna_seu_5, group.by = c("seurat_clusters_adj"), label = TRUE) + ggtitle("RNA") 


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3, repr.plot.res = 200)
FeaturePlot(rna_seu_5, features = c('Eef1a1', 'Fth1'), label = TRUE) 


In [ ]:
library(enrichR)
dbname = as.data.table(listEnrichrDbs())
dbname[grep('reactome', libraryName, ignore.case = T), ]


In [ ]:
dbname[grep('metab', libraryName, ignore.case = T), ]


In [ ]:
dbname[grep('Transcription_Factor_PPIs', libraryName, ignore.case = T), ]


In [ ]:
rna_seu_5@active.ident = rna_seu_5$seurat_clusters_adj

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 6, repr.plot.res = 300)
path_list = DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4', '5'), balanced = F, return.gene.list = T, p.val.cutoff = 0.05, enrich.database = "HMDB_Metabolites", max.genes = 250, num.pathway = 20)
path_list = as.data.table(path_list$pos)
path_list = path_list[HMDB_Metabolites.Adjusted.P.value < 0.05, ]
path_list


In [ ]:
path_list = as.data.table(path_list$pos)
path_list = path_list[HMDB_Metabolites.Adjusted.P.value < 0.05, ]
path_list

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300)
up_6vs1 = DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4', '5'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4', '5'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 5.5, repr.plot.res = 300)
dn_6vs1 = DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('1', '2', '3', '4', '5'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('1', '2', '3', '4', '5'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300)
up_6vs7 = DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('7'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('7'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300)
dn_6vs7 = DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('7'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('7'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 3, repr.plot.res = 300)
up_6vsall = DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4', '5', '7'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4', '5', '7'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 300)
dn_6vsall = DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('1', '2', '3', '4', '5', '7'), return.gene.list = T, balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)
DEenrichRPlot(rna_seu_5, ident.2 = c('6'), ident.1 = c('1', '2', '3', '4', '5', '7'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 50, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3, repr.plot.res = 200)
feat = str_to_title(c('NDUFA4L2', 'GAPDH', 'EEF1A1'))
DotPlot(rna_seu_5, features = feat)
#FeaturePlot(rna_seu_5, features = feat, label = TRUE) 


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3, repr.plot.res = 200)
feat = str_to_title(c('LARS2', 'UBR5', 'HUWE1', 'MID1', 'HEXB'))
DotPlot(rna_seu_5, features = feat)
#FeaturePlot(rna_seu_5, features = feat, label = TRUE) 


In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 6, repr.plot.res = 300)
DEenrichRPlot(rna_seu_5, ident.1 = c('5', '6'), ident.2 = c('7'), balanced = F, enrich.database = "HMDB_Metabolites", max.genes = 250, num.pathway = 20, logfc.threshold = 1, p.val.cutoff = 0.05)

In [ ]:
#PPI_Hub_Proteins GeneSigDB KEGG_2013 Reactome_2013 GO_Cellular_Component_2015 GO_Biological_Process_2015 CellMarker_2024
options(repr.plot.width = 10, repr.plot.height = 3, repr.plot.res = 300)
gene_list = DEenrichRPlot(rna_seu_5, ident.1 = c('6'), ident.2 = c('1', '2', '3', '4'), balanced = F, enrich.database = "Transcription_Factor_PPIs", return.gene.list = T,
              p.val.cutoff = 0.01,  max.genes = 50)
# DEenrichRPlot(object = ctrl.test, ident.1 = "Basal_1",ident.2 = "Basal_2",enrich.database = "BP",max.genes = 50)

In [ ]:
gene_list = as.data.table(gene_list)

In [ ]:
feat_sel = c('GAPDH', 'RPL5', 'RPL32', 'RPL10', 'RPL11', 'RPL10A', 'RPL8', 'RPL9', 'RPS4X', 'RPS16', 'RPL18A', 'RPS3', 'RPL13', 'RPS2', 'RPL15', 'RPL18', 'RPL19', 'RPL21', 'RPS8', 'RPL23', 'RPL23A', 'EEF1A1', 'RPS28', 'RPL27', 'RPS20', 'RPS23')
feat_sel = geneId[feat_sel, Mouse.gene.name]
feat_sel = c(feat_sel[feat_sel != ''], 'Gapdh')
feat_sel

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 3, repr.plot.res = 200)
rna_seu_5_copy = ScaleData(rna_seu_5, features = feat_sel , assay = 'RNA') 
ret = DoHeatmap(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj', assay = 'RNA', slot = 'scale.data') + 
scale_fill_gradientn(colors = c("blue", "white", "red"))
ret


In [ ]:
options(repr.plot.width = 7, repr.plot.height = 3, repr.plot.res = 200)
DotPlot(rna_seu_5_copy, features = feat_sel, group.by = 'seurat_clusters_adj') + 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = .5))